In [1]:
# !jupyter notebook --NotebookApp.iopub_data_rate_limit=10000000

In [2]:
import pandas as pd
import numpy as np
import json
import re
from datetime import datetime
import rdflib
from rdflib import Graph, URIRef, Literal, Namespace, RDF, BNode, RDFS,XSD
from rdflib.namespace import RDFS,RDF,SDO, XSD
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")

def time_convert(d):
    return datetime.strptime(d, '%B %d, %Y').strftime('%Y-%m-%d')

In [3]:
og_songs = pd.read_csv('original_song_stats.csv')
og_songs = og_songs.drop(['title_url', 'written_by_url','originally_by_url','adaptations'], axis = 1)
og_songs['type'] = 'original'
og_songs

,index,title,written_by,originally_by,covers,type
0,1,Let It Go,"Robert Lopez, Kristen Anderson-Lopez",Idina Menzel,150,original
1,2,All of Me,"John Legend, Toby Gad",John Legend,130,original
2,3,Hello,"Adele [GB], Greg Kurstin",Adele,125,original
3,4,Thinking Out Loud,"Ed Sheeran, Amy Wadge",Ed Sheeran,123,original
4,5,Perfect,Ed Sheeran,Ed Sheeran,120,original
...,...,...,...,...,...,...
10015,10016,Jag och Bobby McGee,Ewert Ljusberg,Ewert Ljusberg,0,original
10016,10017,Komm und tanz,Frank Viehweg,Frank Viehweg,0,original
10017,10018,Was ich nicht habe,Frank Viehweg,Frank Viehweg,0,original
10018,10019,Ein herabgestürzter Engel,Frank Viehweg,Frank Viehweg,0,original


In [4]:
cover_data = []
with open('sendhand_cover.jsonl') as json_file:
    for line in json_file:
        cover_data.append(json.loads(line))
# cover_data
covers = pd.DataFrame(cover_data, columns = ['original_song_id', 'song_name','singer_interlink','release_date'])
covers.sort_values('original_song_id')
covers.reset_index(drop=True, inplace=True)
covers

,original_song_id,song_name,singer_interlink,release_date
0,1,Let It Go,https://secondhandsongs.com/artist/44081,"November 25, 2013"
1,1,Let It Go,https://secondhandsongs.com/artist/73158,"October 22, 2013"
2,1,Let It Go,https://secondhandsongs.com/artist/95312,"January 2, 2014"
3,1,Let It Go,https://secondhandsongs.com/artist/110081,"January 11, 2014"
4,1,Let It Go,https://secondhandsongs.com/artist/77796,"January 24, 2014"
...,...,...,...,...
160640,10016,NaN,NaN,NaN
160641,10016,NaN,NaN,NaN
160642,10016,NaN,NaN,NaN
160643,10016,NaN,NaN,NaN


In [5]:
singer_w_names = []
with open('sendhand_singer_name_url.jsonl') as json_file:
    for line in json_file:
        singer_w_names.append(json.loads(line))
# singer_w_names        
singer_w_names = pd.DataFrame(singer_w_names, 
                          columns = ['singer_name','singer_url'])
singer_w_names.rename(columns = {'singer_url':'singer_interlink'}, inplace = True)
singer_w_names.drop_duplicates
singer_w_names

,singer_name,singer_interlink
0,Claude King,https://secondhandsongs.com/artist/18327
1,Ameera Delandro & Sonic,https://secondhandsongs.com/artist/131440+131442
2,J. Tex,https://secondhandsongs.com/artist/167441
3,Sylvan Esso,https://secondhandsongs.com/artist/177683
4,Bill Nighy,https://secondhandsongs.com/artist/86387
...,...,...
34834,Ayrton Montarroyos,https://secondhandsongs.com/artist/174093
34835,Shirley & Squirrely,https://secondhandsongs.com/artist/154354
34836,Jerry Hadley,https://secondhandsongs.com/artist/57319
34837,Terry Gibbs,https://secondhandsongs.com/artist/23123


In [6]:
cover_songs = pd.merge(covers, singer_w_names, on="singer_interlink", how ='left')
cover_songs.rename(columns={'original_song_id':'index','singer_name':'covered_by','song_name':'title'}, inplace = True)
cover_songs.drop(columns=['singer_interlink'], axis = 1, inplace = True)
cover_songs.drop_duplicates(inplace=True)
cover_songs.reset_index(drop=True, inplace=True)
cover_songs.sort_values('index')
cover_songs['index']= cover_songs['index'].astype(int)
cover_songs

,index,title,release_date,covered_by
0,1,Let It Go,"November 25, 2013",Idina Menzel
1,1,Let It Go,"October 22, 2013",Demi Lovato
2,1,Let It Go,"January 2, 2014",Gardiner Sisters
3,1,Let It Go,"January 11, 2014",Grace Lee
4,1,Let It Go,"January 24, 2014",Caleb Hyles
...,...,...,...,...
111756,10006,Over the Rainbow,1962,NaN
111757,10016,Me & Bobby McGee,May 2018,Gilly & the Girl
111758,10016,Me and Bobby McGee,"November 22, 2019",Hans Hannemann
111759,10016,Me & Bobby McGee,2019,NaN


In [7]:
all_songs = pd.merge(og_songs, cover_songs, on=['index','title'], how='left')
all_songs

,index,title,written_by,originally_by,covers,type,release_date,covered_by
0,1,Let It Go,"Robert Lopez, Kristen Anderson-Lopez",Idina Menzel,150,original,"November 25, 2013",Idina Menzel
1,1,Let It Go,"Robert Lopez, Kristen Anderson-Lopez",Idina Menzel,150,original,"October 22, 2013",Demi Lovato
2,1,Let It Go,"Robert Lopez, Kristen Anderson-Lopez",Idina Menzel,150,original,"January 2, 2014",Gardiner Sisters
3,1,Let It Go,"Robert Lopez, Kristen Anderson-Lopez",Idina Menzel,150,original,"January 11, 2014",Grace Lee
4,1,Let It Go,"Robert Lopez, Kristen Anderson-Lopez",Idina Menzel,150,original,"January 24, 2014",Caleb Hyles
...,...,...,...,...,...,...,...,...
32667,10016,Jag och Bobby McGee,Ewert Ljusberg,Ewert Ljusberg,0,original,NaN,NaN
32668,10017,Komm und tanz,Frank Viehweg,Frank Viehweg,0,original,NaN,NaN
32669,10018,Was ich nicht habe,Frank Viehweg,Frank Viehweg,0,original,NaN,NaN
32670,10019,Ein herabgestürzter Engel,Frank Viehweg,Frank Viehweg,0,original,NaN,NaN


### Singers info

In [8]:
singer_fm_data = []
with open('lastFM_singer.jsonl') as json_file:
    for line in json_file:
        singer_fm_data.append(json.loads(line))
# singer_fm_data        
singers_fm = pd.DataFrame(singer_fm_data, 
                          columns = ['url_lastFM','singerName_lastFM','img_url','tag','externalLink_homepage','externalLink_twitter',
                                     'externalLink_facebook','externalLink_instagram','similar_singers'])
singers_fm.rename(columns = {'singerName_lastFM':'singer_name'}, inplace = True)
singers_fm.drop_duplicates
singers_fm

,url_lastFM,singer_name,img_url,tag,externalLink_homepage,externalLink_twitter,externalLink_facebook,externalLink_instagram,similar_singers
0,https://www.last.fm/music/Marlisa,Marlisa,NaN,"[pop, australian, all, australia]",None,None,None,None,"[{'name': 'Taylor Henderson', 'page_url': 'htt..."
1,https://www.last.fm/music/Ryan+Dolan,Ryan Dolan,NaN,"[eurovision, pop, irish, dance, male vocalists]",None,None,https://www.facebook.com/ryandolansmusic,https://www.instagram.com/ryandolanmusic/,"[{'name': 'Farid Mammadov', 'page_url': 'https..."
2,https://www.last.fm/music/Savannah+Outen,Savannah Outen,NaN,"[pop, boneriffic, female vocalists, rich, part...",None,None,None,None,"[{'name': 'Tiffany Alvord', 'page_url': 'https..."
3,https://www.last.fm/music/Jasmine+Thompson,Jasmine Thompson,NaN,"[pop, female vocalists, boneriffic, british, all]",http://jasminethompsonmusic.com/,https://twitter.com/TantrumJas,https://www.facebook.com/TantrumJas,https://www.instagram.com/jasminethompson/,"[{'name': 'Madilyn Bailey', 'page_url': 'https..."
4,https://www.last.fm/music/Harrison+Craig,Harrison Craig,NaN,"[pop, vocal, australian, australia, adult cont...",https://harrisoncraig.com.au/,https://twitter.com/harrisoncraig,https://www.facebook.com/HarrisonCraigOfficial,https://www.instagram.com/harrisoncraigofficial/,"[{'name': 'Josh Groban', 'page_url': 'https://..."
...,...,...,...,...,...,...,...,...,...
9696,https://www.last.fm/music/Elvira+Nikolaisen,Elvira Nikolaisen,NaN,"[norwegian, pop, singer-songwriter, female voc...",None,None,None,None,"[{'name': 'Unni Wilhelmsen', 'page_url': 'http..."
9697,https://www.last.fm/music/Alan+Lomax,Alan Lomax,NaN,"[folk, blues, country, delta blues, musicologist]",None,None,None,None,"[{'name': 'Blind Willie Johnson', 'page_url': ..."
9698,https://www.last.fm/music/Kristina+Bach,Kristina Bach,NaN,"[schlager, kristina bach, deutsche schlager, d...",http://www.kristinabach.de/,None,None,None,"[{'name': 'Ireen Sheer', 'page_url': 'https://..."
9699,https://www.last.fm/music/Jan+Eggum,Jan Eggum,NaN,"[singer-songwriter, norwegian, viser, folk, no...",http://www.stageway.no/janeggum/,None,https://www.facebook.com/janeggum,None,"[{'name': 'Øystein Dolmen', 'page_url': 'https..."


In [9]:
singer_sh_data = []
with open('sendhand_singer.jsonl') as json_file:
    for line in json_file:
        singer_sh_data.append(json.loads(line))
# singer_sh_data
singers_sh = pd.DataFrame(singer_sh_data, columns = ['singer_name','born_date','born_place'])
singers_sh.drop_duplicates
singers_sh.head()

,singer_name,born_date,born_place
0,Marlisa,"October 1, 1999",Australia
1,Sia,"December 18, 1975",Australia
2,Madilyn Bailey,"September 2, 1992",United States
3,Ed Sheeran,"February 17, 1991",United Kingdom
4,Ryan Dolan,"July 22, 1985",United Kingdom


In [10]:
similar_singers = singers_fm['similar_singers']

dummy_name = []
dummy_page = []
dummy_img = []

for i in range(len(similar_singers)):
    current = similar_singers.iloc[i]
    for j in range(len(current)):
        dummy_name.append(current[j]['name'])
        dummy_page.append(current[j]['page_url'])
        dummy_img.append(current[j]['img_url'])
similar_singers = pd.DataFrame({'singer_name':dummy_name, 'url_lastFM':dummy_page,'img_url':dummy_img})
similar_singers.drop_duplicates(inplace=True)
similar_singers.reset_index(drop=True, inplace=True)
similar_singers.astype(str)
similar_singers.head()

,singer_name,url_lastFM,img_url
0,Taylor Henderson,https://www.last.fm//music/Taylor+Henderson,https://lastfm.freetls.fastly.net/i/u/300x300/...
1,Bonnie Anderson,https://www.last.fm//music/Bonnie+Anderson,https://lastfm.freetls.fastly.net/i/u/300x300/...
2,Reigan,https://www.last.fm//music/Reigan,https://lastfm.freetls.fastly.net/i/u/300x300/...
3,Farid Mammadov,https://www.last.fm//music/Farid+Mammadov,https://lastfm.freetls.fastly.net/i/u/300x300/...
4,Andrius Pojavis,https://www.last.fm//music/Andrius+Pojavis,https://lastfm.freetls.fastly.net/i/u/300x300/...


In [11]:
singers_matching = pd.merge(singers_fm, singers_sh, on="singer_name",how='inner')
# singers_matching = singers_matching.drop(columns=["similar_singers","tag"], axis = 1)
# singers_matching.drop_duplicates(inplace=True)
singers_matching.astype(str)
singers_matching.head()

,url_lastFM,singer_name,img_url,tag,externalLink_homepage,externalLink_twitter,externalLink_facebook,externalLink_instagram,similar_singers,born_date,born_place
0,https://www.last.fm/music/Marlisa,Marlisa,NaN,"[pop, australian, all, australia]",None,None,None,None,"[{'name': 'Taylor Henderson', 'page_url': 'htt...","October 1, 1999",Australia
1,https://www.last.fm/music/Marlisa,Marlisa,NaN,"[pop, australian, all, australia]",None,None,None,None,"[{'name': 'Taylor Henderson', 'page_url': 'htt...","October 1, 1999",Australia
2,https://www.last.fm/music/Marlisa,Marlisa,NaN,"[pop, australian, all, australia]",None,None,None,None,"[{'name': 'Taylor Henderson', 'page_url': 'htt...","October 1, 1999",Australia
3,https://www.last.fm/music/Marlisa,Marlisa,NaN,"[pop, australian, all, australia]",None,None,None,None,"[{'name': 'Taylor Henderson', 'page_url': 'htt...","October 1, 1999",Australia
4,https://www.last.fm/music/Marlisa,Marlisa,NaN,"[pop, australian, all, australia]",None,None,None,None,"[{'name': 'Taylor Henderson', 'page_url': 'htt...","October 1, 1999",Australia


In [12]:
singers = pd.merge(singers_matching, similar_singers, on=['singer_name'],how='left')
singers.drop(['img_url_x','url_lastFM_y'], axis = 1,inplace=True)
singers.rename(columns={'url_lastFM_x':'url_lastFM','img_url_y':'img_url'}, inplace=True)
singers.replace(to_replace=[None], value=np.nan, inplace=True)
# singers.drop_duplicates(inplace=True)
singers.head()

,url_lastFM,singer_name,tag,externalLink_homepage,externalLink_twitter,externalLink_facebook,externalLink_instagram,similar_singers,born_date,born_place,img_url
0,https://www.last.fm/music/Marlisa,Marlisa,"[pop, australian, all, australia]",NaN,NaN,NaN,NaN,"[{'name': 'Taylor Henderson', 'page_url': 'htt...","October 1, 1999",Australia,https://lastfm.freetls.fastly.net/i/u/300x300/...
1,https://www.last.fm/music/Marlisa,Marlisa,"[pop, australian, all, australia]",NaN,NaN,NaN,NaN,"[{'name': 'Taylor Henderson', 'page_url': 'htt...","October 1, 1999",Australia,https://lastfm.freetls.fastly.net/i/u/300x300/...
2,https://www.last.fm/music/Marlisa,Marlisa,"[pop, australian, all, australia]",NaN,NaN,NaN,NaN,"[{'name': 'Taylor Henderson', 'page_url': 'htt...","October 1, 1999",Australia,https://lastfm.freetls.fastly.net/i/u/300x300/...
3,https://www.last.fm/music/Marlisa,Marlisa,"[pop, australian, all, australia]",NaN,NaN,NaN,NaN,"[{'name': 'Taylor Henderson', 'page_url': 'htt...","October 1, 1999",Australia,https://lastfm.freetls.fastly.net/i/u/300x300/...
4,https://www.last.fm/music/Marlisa,Marlisa,"[pop, australian, all, australia]",NaN,NaN,NaN,NaN,"[{'name': 'Taylor Henderson', 'page_url': 'htt...","October 1, 1999",Australia,https://lastfm.freetls.fastly.net/i/u/300x300/...


In [13]:
# cleaned_singers = singers[['singer_name','born_date','born_place','externalLink_homepage','externalLink_twitter','externalLink_facebook']]
# cleaned_singers.drop_duplicates(inplace = True)
# cleaned_singers.reset_index(drop=True, inplace=True)
# cleaned_singers
# cleaned_singers.to_csv('cleaned_singers.csv', header= True, index=False)

### Awards

In [14]:
wiki_sh_award = pd.read_csv('blocked.csv')
wiki_sh_award['wiki_id'] = 'http://www.wikidata.org/entity/' + wiki_sh_award[' wiki_nominated_ID'].str.strip()
wiki_sh_award.rename(columns = {'secondhandsong_ID':'index'}, inplace = True)
wiki_sh_award['index'] = wiki_sh_award['index'].astype(int)
wiki_sh_award

,index,wiki_nominated_ID,wiki_id
0,3,Q57836536,http://www.wikidata.org/entity/Q57836536
1,204,Q108805763,http://www.wikidata.org/entity/Q108805763
2,6,Q28132505,http://www.wikidata.org/entity/Q28132505
3,156,Q15272117,http://www.wikidata.org/entity/Q15272117
4,7,Q15275920,http://www.wikidata.org/entity/Q15275920
5,8,Q14661781,http://www.wikidata.org/entity/Q14661781
6,165,Q20962306,http://www.wikidata.org/entity/Q20962306
7,11,Q16326586,http://www.wikidata.org/entity/Q16326586
8,97,Q18786541,http://www.wikidata.org/entity/Q18786541
9,26,Q18559542,http://www.wikidata.org/entity/Q18559542


In [15]:
wiki_award = pd.read_csv('nominated_or_received_awards3.csv')
wiki_award.rename(columns = {'song':'wiki_id'}, inplace = True)
for i in ['nomination_year','award_year']:
    wiki_award[i] = wiki_award[i].fillna(-1)
    wiki_award[i] = wiki_award[i].astype(int)
    wiki_award[i] = wiki_award[i].astype(str)
    wiki_award[i] = wiki_award[i].replace('-1', np.nan)
#wiki_award['wiki_id'] = wiki_award['wiki_id'].str.split('/').str[-1]
wiki_award

,wiki_id,songLabel,release_date,performerLabel,nominated,nominatedLabel,nomination_year,award,awardLabel,award_year
0,http://www.wikidata.org/entity/Q17628069,Q17628069,2012-01-01T00:00:00Z,C2C,NaN,NaN,NaN,http://www.wikidata.org/entity/Q16682573,Victoire du vidéo-clip,NaN
1,http://www.wikidata.org/entity/Q16323774,Intelligent,2012-01-01T00:00:00Z,Raske Penge,http://www.wikidata.org/entity/Q108712768,Q108712768,NaN,NaN,NaN,NaN
2,http://www.wikidata.org/entity/Q109659406,Q109659406,2012-01-15T00:00:00Z,Svetlana Loboda,http://www.wikidata.org/entity/Q25442872,YUNA,NaN,NaN,NaN,NaN
3,http://www.wikidata.org/entity/Q858020,Frontside Ollie,2012-01-16T00:00:00Z,Robin Packalen,NaN,NaN,NaN,http://www.wikidata.org/entity/Q3194212,platinum record,NaN
4,http://www.wikidata.org/entity/Q3878196,Non è l'inferno,2012-02-15T00:00:00Z,Emma Marrone,NaN,NaN,NaN,http://www.wikidata.org/entity/Q84321390,Sanremo Music Festival award,NaN
...,...,...,...,...,...,...,...,...,...,...
281,http://www.wikidata.org/entity/Q112774463,Taksi,2021-09-10T00:00:00Z,Kalush,http://www.wikidata.org/entity/Q61373711,YUNA for Best Hip Hop Hit,NaN,NaN,NaN,NaN
282,http://www.wikidata.org/entity/Q108805763,Easy on Me,2021-10-15T00:00:00Z,Adele,NaN,NaN,NaN,http://www.wikidata.org/entity/Q6314109,Juno Award for Video of the Year,NaN
283,http://www.wikidata.org/entity/Q108805763,Easy on Me,2021-10-15T00:00:00Z,Adele,NaN,NaN,NaN,http://www.wikidata.org/entity/Q48816647,Brit Award for British Single of the Year,NaN
284,http://www.wikidata.org/entity/Q115083235,Tiba-Tiba,2022-06-12T00:00:00Z,Quinn Salman,NaN,NaN,NaN,http://www.wikidata.org/entity/Q4777741,Anugerah Musik Indonesia,NaN


In [16]:
award_matching = pd.merge(wiki_award, wiki_sh_award, on="wiki_id")
award_matching.drop(['wiki_id','release_date','performerLabel'], axis = 1,inplace=True)
award_matching

,songLabel,nominated,nominatedLabel,nomination_year,award,awardLabel,award_year,index,wiki_nominated_ID
0,Summertime Sadness,NaN,NaN,NaN,http://www.wikidata.org/entity/Q41254,Grammy Awards,NaN,115,Q951715
1,I See Fire,http://www.wikidata.org/entity/Q919999,Satellite Award for Best Original Song,NaN,NaN,NaN,NaN,156,Q15272117
2,L'essenziale,NaN,NaN,NaN,http://www.wikidata.org/entity/Q84321390,Sanremo Music Festival award,2013,4586,Q5621290
3,Stolen Dance,NaN,NaN,NaN,http://www.wikidata.org/entity/Q209000,1 Live Krone,NaN,852,Q16011758
4,Young and Beautiful,http://www.wikidata.org/entity/Q428808,Grammy Award for Best Song Written for Visual ...,2013,NaN,NaN,NaN,81,Q12252332
...,...,...,...,...,...,...,...,...,...
81,Montero (Call Me by Your Name),NaN,NaN,NaN,http://www.wikidata.org/entity/Q372123,MTV Video Music Award for Best Visual Effects,2021,609,Q105865170
82,Montero (Call Me by Your Name),NaN,NaN,NaN,http://www.wikidata.org/entity/Q915665,MTV Video Music Award for Best Direction,2021,609,Q105865170
83,Montero (Call Me by Your Name),NaN,NaN,NaN,http://www.wikidata.org/entity/Q1065651,MTV Video Music Award for Video of the Year,2021,609,Q105865170
84,Easy on Me,NaN,NaN,NaN,http://www.wikidata.org/entity/Q6314109,Juno Award for Video of the Year,NaN,204,Q108805763


In [17]:
og_songs_award = pd.merge(all_songs, award_matching, on="index", how ='left')
# award_matching = award_matching[['songLabel','nominated','nominatedLabel','nomination_year',
#                                  'award','awardLabel','award_year']]
og_songs_award.drop_duplicates(inplace = True)
og_songs_award.reset_index(drop=True, inplace=True)
og_songs_award['index'].astype(str)
og_songs_award

,index,title,written_by,originally_by,covers,type,release_date,covered_by,songLabel,nominated,nominatedLabel,nomination_year,award,awardLabel,award_year,wiki_nominated_ID
0,1,Let It Go,"Robert Lopez, Kristen Anderson-Lopez",Idina Menzel,150,original,"November 25, 2013",Idina Menzel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Let It Go,"Robert Lopez, Kristen Anderson-Lopez",Idina Menzel,150,original,"October 22, 2013",Demi Lovato,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,Let It Go,"Robert Lopez, Kristen Anderson-Lopez",Idina Menzel,150,original,"January 2, 2014",Gardiner Sisters,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,Let It Go,"Robert Lopez, Kristen Anderson-Lopez",Idina Menzel,150,original,"January 11, 2014",Grace Lee,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,Let It Go,"Robert Lopez, Kristen Anderson-Lopez",Idina Menzel,150,original,"January 24, 2014",Caleb Hyles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33350,10016,Jag och Bobby McGee,Ewert Ljusberg,Ewert Ljusberg,0,original,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33351,10017,Komm und tanz,Frank Viehweg,Frank Viehweg,0,original,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33352,10018,Was ich nicht habe,Frank Viehweg,Frank Viehweg,0,original,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33353,10019,Ein herabgestürzter Engel,Frank Viehweg,Frank Viehweg,0,original,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
dates = og_songs_award['release_date']
dates = dates.replace({'nan': np.nan})
dates = dates.replace({' ': np.nan})
dates = dates.replace({'Unreleased': np.nan})

corrected_dates = []
for date in dates:
    if not pd.isna(date):
        if len(date.split()) == 1:
            corrected_dates.append('January 1, ' + date)
        elif len(date.split()) == 2:
            corrected_dates.append(date.split()[0] + ' 1, ' + date.split()[1])
        else:
            current_day = date.split()[1]
            for suffix in ['st', 'nd', 'rd','th']:
                current_day = current_day.rstrip(suffix)
            corrected_dates.append(date.split()[0] + ' ' + current_day + ' ' + date.split()[2])
    else:
        corrected_dates.append('')
        
og_songs_award['release_date'] = corrected_dates
# print(og_songs_award['release_date'])
for i in range(len(og_songs_award)):
    try:
        og_songs_award['release_date'][i] = time_convert(og_songs_award['release_date'][i])
    except:
        continue
og_songs_award['release_date'] = og_songs_award['release_date'].replace(r'', np.nan, regex=True)

In [19]:
for i in range(len(og_songs_award)):
    if pd.notna(og_songs_award['originally_by'][i]) & pd.notna(og_songs_award['covered_by'][i]):        
        if og_songs_award['originally_by'][i] == og_songs_award['covered_by'][i]:
            og_songs_award['type'][i] = 'original'
            
        else:
            og_songs_award['type'][i] = 'cover' 

    elif pd.notna(og_songs_award['originally_by'][i]):
        og_songs_award['type'][i] = 'original'

    elif pd.notna(og_songs_award['covered_by'][i]):
        og_songs_award['type'][i] = 'cover'

In [20]:
# og_songs_award.drop(['originally_by'], axis=1, inplace = True)
og_songs_award.rename(columns ={'covered_by':'sung_by'}, inplace = True)
og_songs_award

,index,title,written_by,originally_by,covers,type,release_date,sung_by,songLabel,nominated,nominatedLabel,nomination_year,award,awardLabel,award_year,wiki_nominated_ID
0,1,Let It Go,"Robert Lopez, Kristen Anderson-Lopez",Idina Menzel,150,original,2013-11-25,Idina Menzel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Let It Go,"Robert Lopez, Kristen Anderson-Lopez",Idina Menzel,150,cover,2013-10-22,Demi Lovato,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,Let It Go,"Robert Lopez, Kristen Anderson-Lopez",Idina Menzel,150,cover,2014-01-02,Gardiner Sisters,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,Let It Go,"Robert Lopez, Kristen Anderson-Lopez",Idina Menzel,150,cover,2014-01-11,Grace Lee,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,Let It Go,"Robert Lopez, Kristen Anderson-Lopez",Idina Menzel,150,cover,2014-01-24,Caleb Hyles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33350,10016,Jag och Bobby McGee,Ewert Ljusberg,Ewert Ljusberg,0,original,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33351,10017,Komm und tanz,Frank Viehweg,Frank Viehweg,0,original,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33352,10018,Was ich nicht habe,Frank Viehweg,Frank Viehweg,0,original,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33353,10019,Ein herabgestürzter Engel,Frank Viehweg,Frank Viehweg,0,original,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
og_songs_award['formatted_title'] = og_songs_award['title'] 
identifiers = []
for i in range(len(og_songs_award)):
    formatted_title = re.sub("[^a-zA-Z]", "_", og_songs_award['title'][i])
    identifiers.append(formatted_title)
og_songs_award['formatted_title'] = identifiers
og_songs_award.drop_duplicates(inplace = True)
og_songs_award

,index,title,written_by,originally_by,covers,type,release_date,sung_by,songLabel,nominated,nominatedLabel,nomination_year,award,awardLabel,award_year,wiki_nominated_ID,formatted_title
0,1,Let It Go,"Robert Lopez, Kristen Anderson-Lopez",Idina Menzel,150,original,2013-11-25,Idina Menzel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Let_It_Go
1,1,Let It Go,"Robert Lopez, Kristen Anderson-Lopez",Idina Menzel,150,cover,2013-10-22,Demi Lovato,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Let_It_Go
2,1,Let It Go,"Robert Lopez, Kristen Anderson-Lopez",Idina Menzel,150,cover,2014-01-02,Gardiner Sisters,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Let_It_Go
3,1,Let It Go,"Robert Lopez, Kristen Anderson-Lopez",Idina Menzel,150,cover,2014-01-11,Grace Lee,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Let_It_Go
4,1,Let It Go,"Robert Lopez, Kristen Anderson-Lopez",Idina Menzel,150,cover,2014-01-24,Caleb Hyles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Let_It_Go
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33350,10016,Jag och Bobby McGee,Ewert Ljusberg,Ewert Ljusberg,0,original,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Jag_och_Bobby_McGee
33351,10017,Komm und tanz,Frank Viehweg,Frank Viehweg,0,original,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Komm_und_tanz
33352,10018,Was ich nicht habe,Frank Viehweg,Frank Viehweg,0,original,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Was_ich_nicht_habe
33353,10019,Ein herabgestürzter Engel,Frank Viehweg,Frank Viehweg,0,original,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ein_herabgest_rzter_Engel


In [22]:
# #all songs
# cleaned_songs = og_songs_award[['title','sung_by','type','covers','release_date']]
# cleaned_songs.drop_duplicates(inplace = True)
# cleaned_songs.reset_index(drop=True, inplace=True)
# cleaned_songs
# cleaned_songs.to_csv('cleaned_songs.csv', header= True, index=False)

In [23]:
# cleaned_award = pd.DataFrame()
# cleaned_award['title'] = pd.concat([og_songs_award['awardLabel'], og_songs_award['nominatedLabel']], sort=False)
# cleaned_award['year'] = pd.concat([og_songs_award['award_year'], og_songs_award['nomination_year']], sort=False)
# cleaned_award = cleaned_award.drop_duplicates().dropna()
# cleaned_award.reset_index(drop=True, inplace=True)
# cleaned_award
# cleaned_award.to_csv('clean_award.csv', header = True, index=None)

### KG Entities

In [24]:
DC =  Namespace('http://purl.org/dc/elements/1.1/')
XSD = Namespace('http://www.w3.org/2001/XMLSchema#')
RDFS = Namespace('http://www.w3.org/2000/01/rdf-schema#')
WIKIDATA = Namespace("https://wikidata.org/")
SCHEMA = Namespace("https://schema.org/")
EX = Namespace('http://dsci558.org/myprojectnamespace#')

g = Graph()
g.bind('dc', DC)
g.bind('xsd', XSD)
g.bind('rdfs', RDFS)
g.bind('ex', EX)
g.bind('wiki', WIKIDATA)
g.bind("schema", SCHEMA)

In [25]:
# songs
for i in range(len(og_songs_award)):
    if og_songs_award['type'][i] == 'original':        
        song_uri = URIRef(EX[og_songs_award['formatted_title'][i]+str(og_songs_award['index'][i])+ '_original'])
        g.add((song_uri, RDF.type, SCHEMA.original_song))
        song_title = Literal(og_songs_award['title'][i], lang='en')
        g.add((song_uri, RDFS.title, song_title))
    
        #add release_date
        release_date = np.nan
        if pd.notna(og_songs_award['release_date'][i]):
            release_date = Literal(og_songs_award['release_date'][i], datatype=XSD['date'])
            g.add((song_uri, RDFS.release_date, release_date))
            
        #add writers
        writers = np.nan
        if pd.notna(og_songs_award['written_by'][i]):
            writers = Literal(og_songs_award['written_by'][i], lang='en')
            g.add((song_uri, RDFS.has_writers, writers))
        
        #add number of covers
        n_covers = Literal(og_songs_award['covers'][i], datatype=XSD['integer'])
        g.add((song_uri, RDFS.covers, n_covers))
    
        #add singers
        singer = np.nan
        if pd.notna(og_songs_award['sung_by'][i]):
            revised_name = re.sub("[^a-zA-Z]", "_", og_songs_award['sung_by'][i])
            singer = Literal(revised_name, lang='en')
            singer_uri = URIRef(EX[revised_name])
            g.add((singer_uri, RDF.type, SCHEMA.singer))

            singer_title = Literal(og_songs_award['sung_by'][i], lang='en')
            g.add((singer_uri, RDFS.title, singer_title))
            g.add((song_uri, DC['sung_by'], singer_uri))
            
    if og_songs_award['type'][i] == 'cover':
        song_uri = URIRef(EX[og_songs_award['formatted_title'][i]+str(i)])
        g.add((song_uri, RDF.type, SCHEMA.cover_song))
        song_title = Literal(og_songs_award['title'][i], lang='en')
        g.add((song_uri, RDFS.title, song_title))
        
        #add release_date
        release_date = np.nan
        if pd.notna(og_songs_award['release_date'][i]):
            release_date = Literal(og_songs_award['release_date'][i], datatype=XSD['date'])
            g.add((song_uri, RDFS.release_date, release_date))        

        #add singers
        singer = np.nan
        if pd.notna(og_songs_award['sung_by'][i]):
            revised_name = re.sub("[^a-zA-Z]", "_", og_songs_award['sung_by'][i])
            singer = Literal(revised_name, lang='en')
            singer_uri = URIRef(EX[revised_name])
            g.add((singer_uri, RDF.type, SCHEMA.singer))

            singer_title = Literal(og_songs_award['sung_by'][i], lang='en')
            g.add((singer_uri, RDFS.title, singer_title))
            g.add((song_uri, DC['sung_by'], singer_uri))    

        g.add((song_uri, DC['is_a_cover_of'] , URIRef(EX[og_songs_award['formatted_title'][i]+ str(og_songs_award['index'][i])+ '_original'])))
 
    
            
    award_title = np.nan
    if pd.notna(og_songs_award['award'][i]):
        award_uri = URIRef(og_songs_award['award'][i])
        award_title = Literal(og_songs_award['awardLabel'][i], lang='en')
        award_year = Literal(og_songs_award['award_year'][i], datatype=XSD.year)
        g.add((award_uri, RDF.type, WIKIDATA.award))
        g.add((award_uri, RDFS.title, award_title))
        g.add((URIRef(EX[og_songs_award['formatted_title'][i]+ str(og_songs_award['index'][i])+ '_original']), DC['won'] , award_uri))
        
        award_year = np.nan
        if pd.notna(og_songs_award['award_year'][i]):
            award_year = Literal(og_songs_award['award_year'][i], datatype=XSD.year)
            g.add((award_uri, RDFS.year , award_year))
    
    award_title = np.nan
    if pd.notna(og_songs_award['nominated'][i]):
        award_uri = URIRef(og_songs_award['nominated'][i])
        award_title = Literal(og_songs_award['nominatedLabel'][i], lang='en')
        g.add((award_uri, RDF.type, WIKIDATA.award))
        g.add((award_uri, RDFS.title, award_title))        
        g.add((URIRef(EX[og_songs_award['formatted_title'][i]+ str(og_songs_award['index'][i])+ '_original']), DC['nominated'] , award_uri))
        
        award_year = np.nan
        if pd.notna(og_songs_award['nomination_year'][i]):
            award_year = Literal(og_songs_award['nomination_year'][i], datatype=XSD.year)
            g.add((award_uri, RDFS.year , award_year))

In [26]:
# singers
for i in range(len(singers)):
    revised_name = re.sub("[^a-zA-Z]", "_", singers['singer_name'][i])
#     revised_name = singers['singer_name'][i].replace(' ', '').replace('\"', '').replace('(','')
    singer_uri = URIRef(EX[revised_name])
    singer_title = Literal(singers['singer_name'][i], lang='en')
    birth_date = Literal(singers['born_date'][i], lang='en')
    
    birth_place = np.nan
    if pd.notna(singers['born_place'][i]):
        birth_place = Literal(singers['born_place'][i], lang='en')
        g.add((singer_uri, RDFS.birth_place, birth_place))
        
    current_tag = singers.iloc[i]['tag']
    for j in range(len(current_tag)):
        tag = Literal(current_tag[j], lang='en')
        g.add((singer_uri, RDFS.tag, tag))

    current = singers.iloc[i]['similar_singers']
    for k in range(len(current)):
#         print(current[k]['name'])
        revised_name = re.sub("[^a-zA-Z]", "_", current[k]['name'])
        similar_singers_uri = URIRef(EX[revised_name])
        similar_singers_title = Literal(current[k]['name'], lang='en')
        g.add((similar_singers_uri, RDF.type, SCHEMA.singer))
        g.add((similar_singers_uri, RDFS.title, similar_singers_title))
        g.add((singer_uri, DC['similar'], similar_singers_uri))
        g.add((similar_singers_uri, DC['similar'], singer_uri))
        
    g.add((singer_uri, RDF.type, SCHEMA.singer))
    g.add((singer_uri, RDFS.title, singer_title))
    g.add((singer_uri, RDFS.birth_date, birth_date))
    
    singer_webpage = np.nan
    if pd.notna(singers['externalLink_homepage'][i]):
        singer_webpage = Literal(singers['externalLink_homepage'][i])
        g.add((singer_uri, RDFS.webpage, singer_webpage))
    
    singer_tt = np.nan
    if pd.notna(singers['externalLink_twitter'][i]):
        singer_tt = Literal(singers['externalLink_twitter'][i])
        g.add((singer_uri, RDFS.twitter, singer_tt))
        
    singer_fb = np.nan
    if pd.notna(singers['externalLink_facebook'][i]):
        singer_fb = Literal(singers['externalLink_facebook'][i])
        g.add((singer_uri, RDFS.facebook, singer_fb))
        
    singer_fb = np.nan
    if pd.notna(singers['externalLink_instagram'][i]):
        singer_ig = Literal(singers['externalLink_instagram'][i])
        g.add((singer_uri, RDFS.instagram, singer_ig))

In [30]:
# print(g.serialize(format='ttl'))

In [28]:
g.serialize('data.ttl', format="ttl") 

<Graph identifier=N66029435a6da4eed928611ba44d649a7 (<class 'rdflib.graph.Graph'>)>

In [29]:
# # songs
# for i in range(len(og_songs_award)):
#     revised_name = re.sub("[^a-zA-Z]", "_", og_songs_award['title'][i]) +'_'+ og_songs_award['release_date'][i].astype(str)
#     song_uri = URIRef(EX[revised_name])
#     g.add((song_uri, RDF.type, SCHEMA.song))
    
#     song_title = Literal(og_songs_award['title'][i], lang='en')
#     g.add((song_uri, RDFS.title, song_title))
    
#     writers = np.nan
#     if pd.notna(og_songs_award['written_by'][i]):
#         writers = Literal(og_songs_award['written_by'][i], lang='en')
#         g.add((song_uri, RDFS.has_writers, writers))
    
#     n_covers = Literal(og_songs_award['covers'][i], datatype=XSD['integer'])
#     g.add((song_uri, RDFS.covers, n_covers))   
    
# #     n_adaptations = Literal(og_songs_award['adaptations'][i], datatype=XSD['integer'])
# #     g.add((song_uri, RDFS.adaptations, n_adaptations))
    
#     originally_by = np.nan
#     covered_by = np.nan
#     release_date = np.nan
#     if pd.notna(og_songs_award['originally_by'][i]) & pd.notna(og_songs_award['covered_by'][i]):        
#         if og_songs_award['originally_by'][i] == og_songs_award['covered_by'][i]:
# #             originally_by = Literal(og_songs_award['originally_by'][i], lang='en')
#             revised_name = re.sub("[^a-zA-Z]", "_", og_songs_award['originally_by'][i])
#             originally_by_uri = URIRef(EX[revised_name])
#             g.add((song_uri, DC['originally_by'], originally_by_uri))
            
#             if pd.notna(og_songs_award['release_date'][i]):
#                 release_date = Literal(og_songs_award['release_date'][i], lang='en')
#                 g.add((song_uri, RDFS.ori_release_date, release_date)) 
            
#         else:
# #             covered_by = Literal(og_songs_award['covered_by'][i])            
#             revised_name = re.sub("[^a-zA-Z]", "_", og_songs_award['covered_by'][i])
#             covered_by_uri = URIRef(EX[revised_name])
#             g.add((song_uri, DC['covered_by'], covered_by_uri))

#             if pd.notna(og_songs_award['release_date'][i]):
#                 release_date = Literal(og_songs_award['release_date'][i], lang='en')
# #                 g.add((DC['covered_by'], release_date, DC['covered_by'])) 


#     elif pd.notna(og_songs_award['originally_by'][i]):        
# #         originally_by = Literal(og_songs_award['originally_by'][i], lang='en')
# #         g.add((song_uri, originally_by_uri, originally_by))
#             revised_name = re.sub("[^a-zA-Z]", "_", og_songs_award['originally_by'][i])
#             originally_by_uri = URIRef(EX[revised_name])
#             g.add((song_uri, DC['originally_by'], originally_by_uri))
            
#             if pd.notna(og_songs_award['release_date'][i]):
#                 release_date = Literal(og_songs_award['release_date'][i], lang='en')
#                 g.add((song_uri, DC['originally_by'], release_date ))
#                 g.add((song_uri, RDFS.ori_release_date, release_date))
# #                 g.add((DC['originally_by'], RDFS.release_date, release_date))


#     elif pd.notna(og_songs_award['covered_by'][i]):
# #         covered_by = Literal(og_songs_award['covered_by'][i])
# #         g.add((song_uri, covered_by_uri, covered_by))
#             revised_name = re.sub("[^a-zA-Z]", "_", og_songs_award['covered_by'][i])
#             covered_by_uri = URIRef(EX[revised_name])
#             g.add((song_uri, DC['covered_by'], covered_by_uri))
            
#             if pd.notna(og_songs_award['release_date'][i]):
#                 release_date = Literal(og_songs_award['release_date'][i], lang='en')
# #                 g.add((DC['covered_by'], release_date, DC['covered_by']))        
            
#     award_title = np.nan
#     if pd.notna(og_songs_award['award'][i]):
#         award_uri = URIRef(og_songs_award['award'][i])
#         award_title = Literal(og_songs_award['awardLabel'][i], lang='en')
#         award_year = Literal(og_songs_award['award_year'][i], datatype=XSD.year)
#         g.add((award_uri, RDF.type, WIKIDATA.award))
#         g.add((award_uri, RDFS.title, award_title))
#         g.add((song_uri, DC['won'] , award_uri))
        
#         award_year = np.nan
#         if pd.notna(og_songs_award['award_year'][i]):
#             award_year = Literal(og_songs_award['award_year'][i], datatype=XSD.year)
#             g.add((award_uri, RDFS.year , award_year))
    
#     award_title = np.nan
#     if pd.notna(og_songs_award['nominated'][i]):
#         award_uri = URIRef(og_songs_award['nominated'][i])
#         award_title = Literal(og_songs_award['nominatedLabel'][i], lang='en')
#         g.add((award_uri, RDF.type, WIKIDATA.award))
#         g.add((award_uri, RDFS.title, award_title))        
#         g.add((song_uri, DC['nominated'] , award_uri))
        
#         award_year = np.nan
#         if pd.notna(og_songs_award['nomination_year'][i]):
#             award_year = Literal(og_songs_award['nomination_year'][i], datatype=XSD.year)
#             g.add((award_uri, RDFS.year , award_year))